In [70]:
# Change these variables to the intended area
location_name = 'hagerstown' 

#get zipcodes of intended state/s
from uszipcode import SearchEngine
search = SearchEngine()
data = search.by_state("Maryland",returns=0)
data2 = search.by_state("Pennsylvania", returns=0)

zip_codes1 = [i.zipcode for i in data]
zip_codes1 = [int(zipcode) for zipcode in zip_codes1]

zip_codes2 = [i.zipcode for i in data2]
zip_codes2 = [int(zipcode) for zipcode in zip_codes2]

zip_codes = zip_codes1+zip_codes2

#print(zip_codes)

#set cbg to cluster on and the minimum population (USER INPUT)
core_cbg = '240430006012' 
min_cluster_pop = 5000


In [71]:
# Change these variables to the intended area
location_name = 'hagerstown' 

#get zipcodes of intended state/s
from uszipcode import SearchEngine
# Remove or update the following line if ExtendedBase is not needed or has been renamed
# from sqlalchemy_mate import ExtendedBase

search = SearchEngine()
data = search.by_state("Maryland", returns=0)
data2 = search.by_state("Pennsylvania", returns=0)

zip_codes1 = [i.zipcode for i in data]
zip_codes1 = [int(zipcode) for zipcode in zip_codes1]

zip_codes2 = [i.zipcode for i in data2]
zip_codes2 = [int(zipcode) for zipcode in zip_codes2]

zip_codes = zip_codes1 + zip_codes2

#print(zip_codes)

#set cbg to cluster on and the minimum population (USER INPUT)
core_cbg = '240430006012' 
min_cluster_pop = 5000

In [72]:
import geopandas as gpd
import numpy as np

import pandas as pd

import networkx as nx
import json
import pickle

In [73]:
#Read safegraph files
filename = location_name + '.csv'

try:
    
    df = pd.read_csv(filename)
except:
    datalist = []

    with pd.read_csv('patterns.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    df = pd.concat(datalist, axis=0)
    del datalist

    try:
        
        df['poi_cbg'] = df['poi_cbg'].astype('int64')
    except:
        print(df['poi_cbg'])
        pass
    df.to_csv(filename)

In [74]:
cols = ['location_name', 'safegraph_place_id', 'latitude', 'longitude']
filename = location_name + '.pois.csv'


for _, row in df.iterrows():
    if not row['visitor_daytime_cbgs']:
        continue
    
    for visitor_cbg in json.loads(row['visitor_daytime_cbgs']).keys():
        
        if visitor_cbg not in zip_codes:
            zip_codes.append(visitor_cbg)

try:
    poif = pd.read_csv(filename)
except:
    datalist = []

    with pd.read_csv('2021_05_05_03_core_poi.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    poif = pd.concat(datalist, axis=0)
    del datalist

    poif.to_csv(filename)


In [75]:
#Try to avoid this function especially in loops since it takes time
def cbg_geocode(cbg):
    '''Generates the latitude and longitude coordinates for a CBG

    Args:
        cbg (str): CBG ID
        
    Returns:
        pd.Series: ID, Latitude, & Longitude coords for a CBG
    '''
    lat = []
    long = []

   
    for _, poi in df.loc[df['poi_cbg'] == int(cbg)].iterrows():
        poi_info = poif.loc[poif['safegraph_place_id'] == poi['safegraph_place_id']]
        lat.append(poi_info.iloc[0]['latitude'])
        long.append(poi_info.iloc[0]['longitude'])
            
    return pd.Series(data={
        'label': str(cbg),
        'latitude': np.mean(lat),
        'longitude': np.mean(long)
    })

In [76]:
def poi_geocode(poi):
    '''Finds the latitude and longitude coordinates for a POI

    Args:
        poi (pd.Series): Series that contains the SafeGraph data for a specific POI

    Returns:
        pd.Series: Name, Latitude, & Longitude coords for a POI
    '''
    place_id = poi if type(poi) == str else poi['safegraph_place_id']
    poi_data = poif.loc[poif['safegraph_place_id'] == place_id]
    return pd.Series(data={
        'label': place_id,
        'latitude': poi_data['latitude'],
        'longitude': poi_data['longitude']
    })

In [77]:
#function to find distances between cbgs, used in the distance graph
from math import sin, cos, atan2, pi, sqrt
def distance(lat1, long1, lat2, long2):
    lat1 = lat1*pi/180
    long1 = long1*pi/180
    lat2 = lat2*pi/180
    long2 = long2*pi/180

    Radius = 6371
    haversine = sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2) * sin((long2-long1)/2)**2
    c = 2*atan2(sqrt(haversine), sqrt(1-haversine))
    dist = Radius*c
    return dist



In [78]:
#CBG shapefiles, to find them go to https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html , find the year you want and under Download choose
# "FTP Archive by State"


#Make sure shapefiles agree with safegraph by creating the map with POI's included

#2022 Shapefiles
# gdf1 = gpd.read_file('tl_rd22_24_bg/tl_rd22_24_bg.shp')
# gdf2 = gpd.read_file('tl_rd22_42_bg/tl_rd22_42_bg.shp')
# gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))


#2010 Shapefiles use these for safegraph (safegraph data is from 2021 but they use 2010 cbgs)
import geopandas as gpd
import pandas as pd

# Ensure the correct path to the shapefiles
gdf1 = gpd.read_file(r'E:\Dileno Diesease Modelling\Algorithms\tl_2010_24_bg10\tl_2010_24_bg10.shp')
gdf2 = gpd.read_file(r'E:\Dileno Diesease Modelling\Algorithms\tl_2010_42_bg10\tl_2010_42_bg10.shp')
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))





In [79]:
def gen_graph(df, weighted=True):
    '''Reads from a SafeGraph dataframe of a certain location

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location
        weighted (bool, optional): Whether the result should have
            weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs and the edges and weights
            are determined by the visitors to and from a node to other nodes
    '''
    G = nx.Graph()
    regg_count = 0
    for _, row in df.iterrows():
        poi_cbg = str(row['poi_cbg'])
        
        try:

            int_poi = int(float(poi_cbg))
            poi_cbg = str(int_poi)
        except:
            regg_count+=1
            continue
        
        G.add_node(poi_cbg, pois = [])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        weight = row['median_dwell'] if weighted else 1
        
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            #print(poi_cbg, visitor_cbg, num_visitors, weight)
            
            
            if visitor_cbg == poi_cbg:
                continue # Ignore edges that connect and come from the same node
            
            # visitor_latitiude = cbg_geocode(visitor_cbg)[1]
            # visitor_long = cbg_geocode(visitor_cbg)[2]
            
            # poi_latitude = cbg_geocode(poi_cbg)[1]
            # poi_long = cbg_geocode(poi_cbg)[2]

            
            #dist = distance(visitor_latitiude,visitor_long,poi_latitude,poi_long)
            #print(visitor_latitiude,  ' ', visitor_long, ' ', poi_latitude, ' ', poi_long)
            # if (np.isnan(visitor_latitiude) or np.isnan(visitor_long) or np.isnan(poi_latitude) or np.isnan(poi_long)):
            #     continue

            
            
            
            if G.has_edge(visitor_cbg, poi_cbg):
                try:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
                except ZeroDivisionError:
                    continue
            else:
                try:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
                except ZeroDivisionError:
                    continue
            # try:
            #     G[visitor_cbg][poi_cbg] += int(num_visitors)
            # except:
            #     G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors))])
        
        # Remove nodes without any edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)

    UG = G.to_undirected()
    for node in G:
        for node2 in nx.neighbors(G, node):
            if node in nx.neighbors(G, node2):
                UG.edges[node, node2]['weight'] = G.edges[node, node2]['weight'] + G.edges[node2, node]['weight'] 
    
    print('G has %d nodes and %d edges.' % (nx.number_of_nodes(UG), nx.number_of_edges(UG)))
    print("bad data ", regg_count)
    return UG

In [80]:
#distance graph where the weights are divided by the distance of the POI from the seed
def gen_d_graph(df, gdf, weighted=True):
    '''Reads from a SafeGraph dataframe of a certain location

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location
        weighted (bool, optional): Whether the result should have
            weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs and the edges and weights
            are determined by the visitors to and from a node to other nodes
    '''
    G = nx.Graph()
    seed_row = gdf[gdf['GEOID'] == '240430006012']
    seed_latitiude = seed_row['latitude'].iloc[0]
    seed_long = seed_row['longitude'].iloc[0]
    regg_count = 0
    for _, row in df.iterrows():
        poi_cbg = str(row['poi_cbg'])
        
        try:

            int_poi = int(float(poi_cbg))
            poi_cbg = str(int_poi)
        except:
            regg_count+=1
            continue
        
        G.add_node(poi_cbg, pois = [])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        weight = row['median_dwell'] if weighted else 1
        
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            #print(poi_cbg, visitor_cbg, num_visitors, weight)
            
            
            if visitor_cbg == poi_cbg:
                continue # Ignore edges that connect and come from the same node
            
           
            try:
                
                
                poi_row = gdf[gdf['GEOID'] == poi_cbg]
                poi_latitude = poi_row['latitude'].iloc[0]
                poi_long = poi_row['longitude'].iloc[0]
               
            except IndexError:
                regg_count +=1
                continue

            
            
            dist = int(distance(seed_latitiude,seed_long,poi_latitude,poi_long))
            if G.has_edge(visitor_cbg, poi_cbg):
                try:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)/dist
                except ZeroDivisionError:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
                   
            else:
                try:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight)/dist)])
                except ZeroDivisionError:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
                    
           
        
        # Remove nodes without any edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)

    #Change to undirected graph
    UG = G.to_undirected()
    for node in G:
        for node2 in nx.neighbors(G, node):
            if node in nx.neighbors(G, node2):
                UG.edges[node, node2]['weight'] = G.edges[node, node2]['weight'] + G.edges[node2, node]['weight'] 
    
    print('G has %d nodes and %d edges.' % (nx.number_of_nodes(UG), nx.number_of_edges(UG)))
    print("bad data ", regg_count)
    return UG

In [81]:
import pandas as pd

#get population of a cbg
cbg_pops = pd.read_csv(r'E:\Dileno Diesease Modelling\Algorithms\safegraph_cbg_population_estimate.csv', index_col='census_block_group')


def cbg_population(cbg):
    try:
        return int(cbg_pops.loc[int(cbg)].B00002e1)
    except (ValueError, TypeError):
        return 0

    

In [82]:
#get coordinates
gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf['coords'] = [coords[0] for coords in gdf['coords']]
gdf['longitude'] = gdf['coords'].apply(lambda x: x[0])
gdf['latitude'] = gdf['coords'].apply(lambda x: x[1])
print('creating graph')


#To create regular graph
G = gen_graph(df)



#To create distance graph (save this graph, it takes long to create)

# G = gen_d_graph(df,gdf) for distance graph or just load (below)



#Save graph

# with open("dist_graph.pkl", "wb") as file:
#     pickle.dump(G, file)

#Load graph
# with open("dist_graph.pkl", "rb") as file:
#     G = pickle.load(file)



creating graph
G has 4265 nodes and 12286 edges.
bad data  0


In [83]:
#To save graph as csv
import pandas as pd
import numpy as np

def graph_to_csv(UG, filename='adj_graph.csv'):
    #Get adjacency matrix
    adj_matrix = nx.adjacency_matrix(UG)

    array_adj_matrix = adj_matrix.toarray()

    # Create the dataframe
    nodes = list(UG.nodes())
    df_adj_matrix = pd.DataFrame(array_adj_matrix, index=nodes, columns=nodes)

    
    df_adj_matrix.to_csv(filename)

    print('Graph written')




In [84]:
#TEST
cbg_geocode('240430005001')

label        240430005001
latitude        39.644759
longitude      -77.716671
dtype: object

In [85]:
#Get total population
total_cbgs_population = 0
for cbg in df.poi_cbg.drop_duplicates().tolist():
   
    #print(cbg)
    total_cbgs_population += cbg_population(cbg)
total_cbgs_population

3371

In [86]:
def greedy_weight(C, u0, min_pop):
    
    cluster = [ u0 ]
    population = cbg_population(u0)
    it = 1
    while population < min_pop:
        #get all adjacent cbgs of the cluster
        all_adj_cbgs = []
        for i in cluster:
            adj_cbgs = list(C.adj[i])
            
            for j in adj_cbgs:
                if j not in all_adj_cbgs and j not in cluster:
                    all_adj_cbgs.append(j)
        movement_in = 0
        movement_out = 0
        max_movement = 0
        cbg_to_add = 0
        
        
        #Calculate the movement from S to all adjacent cbgs
        for i in all_adj_cbgs:
            current_movement = 0
            
        
            
            for j in cluster:
                try:
                    current_movement += C.adj[i][j]['weight']
                except:
                    pass
            #find the cbg with the greatest movement and add it to S
            if current_movement > max_movement:
                max_movement = current_movement
                cbg_to_add = i
        
        
            
        #Movement checking at each iteration
            
        # for i in cluster:
        #     adj = list(C.adj[i])
        #     for k in adj:
        #         if k in cluster:
        #             movement_in += C.adj[i][k]['weight']/2
        #         else:
        #             movement_out += C.adj[i][k]['weight']
        # ratio = movement_in/(movement_in+movement_out)
        cluster.append(cbg_to_add)
        # print("Iteration #  " , it, end="", flush=True)
        print("  Prev pop  ", population, end="", flush=True)
        population += cbg_population(cbg_to_add)
        print("  New pop  ", population, end="", flush=True)
        # print("  Movement in S  " ,movement_in, end="", flush=True)
        # print("  Move out S  ", movement_out, end="", flush=True)
        # print("  Ratio  ", ratio)
        # print("")
        # it+=1
    
    return cluster, population

            
        
        
    
   

        
                
       

In [87]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import folium
from folium import plugins
from folium import Marker

In [88]:
print('starting algo')
algorithm_result = greedy_weight(G, core_cbg, min_cluster_pop)
print('algo done')
print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
print(algorithm_result[0])
movement_in_cbgs = 0
movement_out_cbgs = 0


for i in algorithm_result[0]:
    adj = list(G.adj[i])
    for j in adj:
        if j in algorithm_result[0]:
            movement_in_cbgs += G.adj[i][j]['weight']/2
        else:
            movement_out_cbgs += G.adj[i][j]['weight']
    

print("IN ", movement_in_cbgs, "OUT", movement_out_cbgs, " Ratio ", movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs) )
  
    
    

starting algo
  Prev pop   59  New pop   137  Prev pop   137  New pop   179  Prev pop   179  New pop   207  Prev pop   207  New pop   284  Prev pop   284  New pop   328  Prev pop   328  New pop   346  Prev pop   346  New pop   393  Prev pop   393  New pop   450  Prev pop   450  New pop   552  Prev pop   552  New pop   592  Prev pop   592  New pop   681  Prev pop   681  New pop   714  Prev pop   714  New pop   790  Prev pop   790  New pop   854  Prev pop   854  New pop   911  Prev pop   911  New pop   950  Prev pop   950  New pop   1016  Prev pop   1016  New pop   1066  Prev pop   1066  New pop   1147  Prev pop   1147  New pop   1216  Prev pop   1216  New pop   1283  Prev pop   1283  New pop   1352  Prev pop   1352  New pop   1381  Prev pop   1381  New pop   1560  Prev pop   1560  New pop   1646  Prev pop   1646  New pop   1695  Prev pop   1695  New pop   1771  Prev pop   1771  New pop   1798  Prev pop   1798  New pop   1951  Prev pop   1951  New pop   2020  Prev pop   2020  New pop   2

In [89]:
import geopandas as gpd
import folium

# Assuming gdf is already defined and contains the CBG geometries
# Example: gdf = gpd.read_file('path_to_your_shapefile.shp')

# Initialize the map object
map = folium.Map(location=[39.6418, -77.7199], zoom_start=12)  # Adjust the location and zoom level as needed

# Surrounded CBGs
cbg_n = {}
# algo_copy is the new S
algo_copy = algorithm_result[0].copy()
for cbg in algorithm_result[0]:
    try:
        row = gdf[gdf['GEOID10'] == cbg].iloc[0]
    except IndexError:
        continue
        
    # Find all neighboring CBGs
    current_neighbours = gdf[gdf.geometry.touches(row['geometry'])]
    print("N ", current_neighbours['GEOID10'])
    for i in current_neighbours['GEOID10']:
        # Count the neighbors in S
        if i in algorithm_result[0]:
            continue

        if i not in cbg_n:
            cbg_n[i] = 1
        else:
            cbg_n[i] += 1

for key in cbg_n:
    # If more than 3 borders are shared with S
    if cbg_n[key] >= 3:
        print('found')
        shape = gdf[gdf['GEOID10'] == key]
        layer = folium.GeoJson(shape, style_function=lambda feature: {
            'fillColor': '#000000', 'color': '#000000', 'fillOpacity': 0.7
        }).add_to(map)
        layer.add_child(folium.Popup(key))
        algo_copy.append(key)

print('saving')
map.save("pagerank_10000.html")






N  1559    240430006022
1576    240430006011
1577    240430005002
1580    240430005003
1581    240430007001
Name: GEOID10, dtype: object
N  1559    240430006022
1580    240430005003
1586    240430112012
1587    240430006012
1607    240430112013
Name: GEOID10, dtype: object
N  1551    240430112011
1557    240430112022
1563    240430111001
1576    240430006011
1580    240430005003
1583    240430001001
1586    240430112012
1598    240430112021
1613    240430112014
Name: GEOID10, dtype: object
N  1524    240430105004
1530    240430008001
1531    240430008002
1533    240430009002
1535    240430009001
1536    240430010014
1537    240430010011
1569    240430010012
1591    240430004003
1592    240430010021
1601    240430008003
1605    240430010023
Name: GEOID10, dtype: object
N  1537    240430010011
1538    240430010013
1544    240430110003
1558    240430006021
1560    240430110001
1561    240430111004
1564    240430109002
1606    240430109004
Name: GEOID10, dtype: object
N  1524    2404301050

In [90]:
#Map final cbg states as animation - NEEDED

movement_in_S = 0
movement_out_S = 0

seed = cbg_geocode('240430006012')

center = (seed['latitude'], seed['longitude'])
map = folium.Map(location=[center[0],center[1]], zoom_start=13)

features = []
for i, cbg in enumerate(algorithm_result[0]):
    
    movement_in_S = 0
    movement_out_S = 0
    shape = gdf[gdf['GEOID10'] == cbg]

    edges = G.edges(cbg)
    for j in edges:
        
        if j[1] in algorithm_result[0]:

            movement_in_S += G[cbg][j[1]]['weight']/2
           
        else:
            movement_out_S += G[cbg][j[1]]['weight']
            
    
    ratio = movement_in_S/(movement_in_S+movement_out_S)

    if ratio >= 0.8:
        fillcolor = '#0000FF'
        color = '#0000FF'
       
    elif ratio >=0.6:
        fillcolor = '#008000'
        color = '#008000'
       
    elif ratio >=0.4:
        fillcolor = '#FFFF00'
        color = '#FFFF00'
       
    elif ratio >= 0.2:
        fillcolor = '#FFA500'
        color = '#FFA500'
        
    elif ratio >= 0:
        fillcolor = '#FF0000'
        color = '#FF0000'
    
    
    # Check if shape is not empty
    if not shape.empty:
        # Convert to EPSG:4326
        shape = shape.to_crs("EPSG:4326")
        # Convert to GeoJSON
        geojson = json.loads(shape.to_json())
        feature = geojson['features'][0]
        # Add a times property to the feature
        feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'D')).isoformat()]
        feature['properties']['style'] = {'fillColor': fillcolor, 'color': color, 'fillOpacity': 0.7}
        features.append(feature)


# Add the existing clusters to the map
existing_cluster_layer = plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT6H',
    add_last_point=True,
    auto_play=False,
    loop=False
)
map.add_child(existing_cluster_layer)

# Style for the black clusters
black_style = {'fillColor': '#000000', 'color': '#000000', 'fillOpacity': 0.7}

# List of GEOID10 codes to be added in black
black_cbgs = [
    "240430002003", "240430003021", "240430001002", "240430001001",
    "240430008001", "240430008002", "240430008003", "240430007003",
    "240430010014", "240430010012", "240430010021", "240430102003"
]

# Add the black clusters directly to the map without a time property
for cbg in black_cbgs:
    shape = gdf[gdf['GEOID10'] == cbg]
    if not shape.empty:
        # Convert to EPSG:4326 if needed
        shape = shape.to_crs(epsg='4326')
        geojson = json.loads(shape.to_json())
        
        # Create the GeoJson object and add it to the map
        folium.GeoJson(
            geojson,
            style_function=lambda x: black_style
        ).add_to(map)

# Save or display the map
map.save("map_with_black_clusters.html")

# # Identify black clusters
# black_clusters = []
# for cbg in G.nodes():
#     if cbg not in algorithm_result[0]:
#         adjacent_cbgs = list(G.adj[cbg])
#         if all(adj_cbg in algorithm_result[0] for adj_cbg in adjacent_cbgs):
#             black_clusters.append(cbg)

# # Debugging: print the number of black clusters to check
# print(f"Number of black clusters identified: {len(black_clusters)}")


# # Create features for black clusters
# black_cluster_features = []
# for cbg in black_clusters:
#     # Check if the gdf contains the cbg
#     if cbg in gdf['GEOID10'].values:
#         shape = gdf[gdf['GEOID10'] == cbg]
#         # Check the current CRS
#         print(f"Current CRS: {gdf.crs}")
#         # Only convert if the CRS is not already EPSG:4326
#         if gdf.crs != 'EPSG:4326':
#             shape = shape.to_crs("EPSG:4326")
#         geojson = json.loads(shape.to_json())
#         # Check if GeoJSON conversion is successful
#         if 'features' in geojson and len(geojson['features']) > 0:
#             feature = geojson['features'][0]
#             feature['properties']['style'] = {'fillColor': '#000000', 'color': '#000000', 'fillOpacity': 0.7}
#             black_cluster_features.append(feature)
#         else:
#             # Print an error message if the GeoJSON conversion fails
#             print(f"GeoJSON conversion failed for CBG: {cbg}")
#     else:
#         # Print an error message if the CBG is not in gdf
#         print(f"CBG not found in GeoDataFrame: {cbg}")

# # Check if any black cluster features were created
# if black_cluster_features:
#     # Add the black cluster features to the map
#     folium.GeoJson(
#         {'type': 'FeatureCollection', 'features': black_cluster_features},
#         style_function=lambda x: x['properties']['style']
#     ).add_to(map)
# else:
#     print("No black cluster features to add to the map.")


In [ ]:
import yaml

# It is assumed that:
#   - algorithm_result has been computed (it is a tuple with [0]: list of CBGs)
#   - G is your graph
#   - The census DataFrame "census_df" is loaded from safegraph_cbg_population_estimate.csv
#   - The helper function was renamed to get_cbg_population (see cell 17)
#
# The helper function: 
# def get_cbg_population(cbg):
#     try:
#         return int(cbg_pops.loc[int(cbg)].B00002e1)
#     except (ValueError, TypeError):
#         return 0

cbg_info_list = []

for cbg in algorithm_result[0]:
    cbg_str = str(cbg)
    # Get the estimated population from the census data using our helper function
    pop_est = cbg_population(cbg)  # make sure get_cbg_population is defined (rename from cbg_population)
    
    # Calculate movement for the CBG based on the graph edges
    movement_in_S = 0
    movement_out_S = 0

    for neighbor in G.adj[cbg]:
        if neighbor in algorithm_result[0]:
            movement_in_S += G.adj[cbg][neighbor]['weight'] / 2
        else:
            movement_out_S += G.adj[cbg][neighbor]['weight']
    
    total_movement = movement_in_S + movement_out_S
    ratio = movement_in_S / total_movement if total_movement > 0 else None

    # Build the entry with estimated population included
    cbg_info_list.append({
        "GEOID10": cbg_str,
        "movement_in": movement_in_S,
        "movement_out": movement_out_S,
        "ratio": ratio,
        "estimated_population": pop_est
    })

with open("cbg_info.yaml", "w", encoding="utf-8") as outfile:
    yaml.dump(cbg_info_list, outfile)

print("cbg_info.yaml generated successfully.")

cbg_info.yaml generated successfully.
